# Testing a DeeProtGO model 

This notebook is aimed to test a DeeProtGO to predict Bilogical Process (BP) Gene Ontology (GO) terms for the *NK* proteins from Eukarya organisms in the CAFA3 benchmark dataset. 

## Requirements

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F


Downloading GitHub repository and setting data directory

In [4]:
!git clone https://github.com/gamerino/DeeProtGO.git
!./DeeProtGO/examples/train_NK_EUKA_BP/modelPreparation.sh

Cloning into 'DeeProtGO'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 111 (delta 4), reused 10 (delta 3), pack-reused 95
Receiving objects: 100% (111/111), 184.86 MiB | 32.26 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Checking out files: 100% (43/43), done.


In [5]:
os.chdir('DeeProtGO/')

In [6]:
from src.sampler import Sampler
from src.dataloader import Dataloader
from src.DNNModel import DNNModel
from src.logger import Logger
from src.DNN import DNN
from src.earlyStop import EarlyStopping

### 1. Setting global parameters

In [7]:
torch.set_num_threads(torch.get_num_threads())
use_GPU = True
if use_GPU & torch.cuda.is_available():
    torch.cuda.manual_seed_all(1)
else:
    use_GPU = False
np.random.seed(1)
torch.manual_seed(1)


In [8]:
res_dir = "examples/train_NK_EUKA_BP/"
dirData = 'data/processed/'

### 2. Setup parameters and model hyperparameters

In [9]:
activFunc = F.elu
optimMethod = torch.optim.Adam
criterion = nn.BCELoss
thresh = 0.2
learning_rate = 0.005
nbatch = 128
pDrop = 0.5

pPSD1 = 0.5
pPSD2 = 0.35
pEmb1 = 0.7
pEmb2 = 0.5
pTaxon1 = 0.7
pTaxon2 = 0.5

pHidden1 = 1
pHidden2 = 0.7

samplingNegPerc = 0.1399 # Of all negatives, we want to take 0.1399*100 of the data

### 3.  Model loading

In [10]:
propOutFile = dirData + "Training/GOTermsPropRel_Euka_BP_train.tab"
nCases = 7180
N_in_1 = 5308
N_in_2 = 1024
N_in_3 = 408

In [11]:
DeeProtGO_Euka_NK_BP = DNNModel(res_dir, propOutFile = propOutFile, nbatch = nbatch, N_in_1 = N_in_1, 
                                N_in_2 = N_in_2, N_in_3 = N_in_3, pN1_1 = pPSD1, pN1_2 = pPSD2, 
                                pN2_1 = pEmb1, pN2_2 = pEmb2, pN3_1 = pTaxon1, pN3_2 = pTaxon2,
                                pNO_1 = pHidden1, pNO_2 = pHidden2, thresh = thresh, pDrop = pDrop, 
                                activFunc = activFunc, optimMethod = optimMethod, criterion = criterion, 
                                learningRate = learning_rate, useGPU = use_GPU)



In [12]:
DeeProtGO_Euka_NK_BP.net.load_state_dict(torch.load(res_dir + 'DeeProtGO_PSD_Emb_Taxon_Euka_BP_NK.pt'))


<All keys matched successfully>

### 4. Data loading


In [13]:
posProteinsFilePath = dirData + "Benchmark/PosEntries_Euka_BP.tab"
negProteinsFilePath = dirData + "Benchmark/NegEntries_Euka_BP.tab"

posNames = np.loadtxt(posProteinsFilePath, delimiter = '\t', dtype = 'str').tolist()
negNames = np.loadtxt(negProteinsFilePath, delimiter = '\t', dtype = 'str').tolist()
targetNames = np.concatenate([posNames,negNames], axis = 0)


In [14]:
PSDFilePath = dirData + "Benchmark/LevSim_BP_Euka.h5"
inData1 = pd.read_hdf(PSDFilePath)
inData1.head()

Entry,A0A075F932,A0A0C5B5G6,A0A0K3AV08,A0A0N9E2K8,A0A0R4IBK5,A0AVF1,A0FGR8,A0FGR9,A0FLQ6,A0JMQ9,A0JPF9,A0MS83,A0MSJ1,A0MWD1,A0PJK1,A0PK84,A1A547,A1A5H6,A1A5V7,A1IGX5,A1L224,A1L2F3,A1XQX0,A1XQX1,A1Z198,A1Z9R4,A1ZB42,A2A432,A2A690,A2AJN7,A2AKQ8,A2ATD1,A2ATU0,A2BE76,A2BID5,A2QD49,A2QIL5,A2QJF9,A2QPN1,A2R3L3,...,Q9ZV88,Q9ZVB5,Q9ZVQ4,Q9ZW11,Q9ZW27,Q9ZW28,Q9ZW29,Q9ZW30,Q9ZW96,T1SFR8,W8DXL4,W8E7I1,A2ACG1,Q8TBZ3,P0CU04,Q92567,P49106,Q01526,P40214,Q8TD55,Q9LNP2,Q7Y219,Q6X9E4,P0CU02,Q7M6Y6,P53227,A5PLN9,O14280,K4DEY3,Q8NAG6,Q8GWB2,Q5SV66,Q8NBF2,Q8TF61,Q6PKX4,P0CU05,Q3TY65,Q96ME1,Q9FHK4,Q8IUR7
AB17B_MOUSE,0.858904,0.884403,0.622609,0.799405,0.0,0.815980,0.678283,0.693583,0.386315,0.757331,0.848704,0.720357,0.379941,0.759456,0.801105,0.900127,0.882703,0.094348,0.898428,0.906502,0.829579,0.851254,0.453039,0.851254,0.580535,0.558861,0.904377,0.657459,0.247769,0.699958,0.775181,0.849979,0.681258,0.899278,0.722057,0.870378,0.645984,0.792605,0.902252,0.870378,...,0.889078,0.903952,0.902677,0.792180,0.902252,0.901402,0.903102,0.902252,0.903952,0.894603,0.770931,0.904377,0.904802,0.809180,0.863153,0.900977,0.899703,0.899703,0.886103,0.836804,0.751381,0.660008,0.848704,0.771356,0.417765,0.897578,0.864003,0.899703,0.893328,0.794305,0.896303,0.889503,0.751381,0.695708,0.886953,0.863153,0.856354,0.724182,0.794730,0.774331
AB17C_MOUSE,0.855504,0.870803,0.628559,0.800255,0.0,0.814705,0.683383,0.697408,0.390140,0.759881,0.850404,0.722907,0.385040,0.759881,0.800680,0.893328,0.878028,0.099873,0.892053,0.894603,0.830004,0.850404,0.459414,0.852529,0.585635,0.565661,0.891203,0.662558,0.254994,0.704207,0.778581,0.849979,0.683808,0.892903,0.725457,0.866978,0.652784,0.793455,0.889503,0.868253,...,0.887803,0.892903,0.892903,0.794730,0.892053,0.894178,0.892903,0.892478,0.894178,0.889503,0.772206,0.891203,0.897153,0.813005,0.858904,0.893328,0.891628,0.891203,0.880153,0.838079,0.752231,0.661283,0.847429,0.772206,0.422864,0.888653,0.861028,0.886953,0.890353,0.798555,0.891628,0.883978,0.754356,0.704632,0.883553,0.858904,0.857629,0.726732,0.795155,0.773056
ABHD4_MOUSE,0.855504,0.861028,0.630259,0.801530,0.0,0.816830,0.682958,0.699108,0.398215,0.762431,0.844029,0.725457,0.390565,0.759881,0.802805,0.888228,0.875903,0.106247,0.883978,0.884403,0.827029,0.851254,0.463238,0.850829,0.589460,0.561836,0.883978,0.661708,0.260943,0.705482,0.778581,0.848279,0.687633,0.886528,0.726307,0.864003,0.651934,0.794305,0.879728,0.865703,...,0.878028,0.884403,0.886953,0.790905,0.888653,0.887803,0.886528,0.888653,0.888228,0.884403,0.772631,0.885678,0.886953,0.812155,0.856354,0.884828,0.885678,0.885253,0.875903,0.838504,0.756906,0.662983,0.847004,0.777306,0.427964,0.883128,0.858054,0.876753,0.883128,0.795580,0.883978,0.879303,0.757756,0.698258,0.878028,0.856354,0.852104,0.727157,0.793030,0.776031
ABHDD_MOUSE,0.857204,0.863578,0.631109,0.801530,0.0,0.818105,0.685508,0.699108,0.398215,0.757756,0.847004,0.725032,0.389715,0.762006,0.804080,0.889078,0.878028,0.104547,0.886528,0.889078,0.830004,0.851679,0.464938,0.851679,0.586910,0.562686,0.884403,0.664258,0.259669,0.707607,0.780280,0.848704,0.685508,0.888228,0.725882,0.866128,0.650659,0.794305,0.881853,0.864003,...,0.881428,0.887803,0.890778,0.796005,0.889503,0.889078,0.888653,0.889078,0.887803,0.885253,0.772206,0.884828,0.890353,0.812580,0.857204,0.883553,0.889503,0.889503,0.879303,0.838929,0.757331,0.666808,0.846154,0.773481,0.429664,0.885678,0.858904,0.878453,0.884828,0.794305,0.884828,0.881853,0.759456,0.698258,0.879303,0.857204,0.855929,0.728432,0.792605,0.779006
ADPGK_MOUSE,0.831279,0.796005,0.647259,0.796430,0.0,0.810880,0.703782,0.711857,0.422439,0.765406,0.827454,0.735657,0.413940,0.764556,0.803230,0.835104,0.834679,0.141521,0.838504,0.833404,0.818530,0.830854,0.487888,0.833404,0.606460,0.584360,0.823629,0.674883,0.289843,0.716532,0.784530,0.829154,0.696983,0.834254,0.736082,0.834679,0.667658,0.792180,0.820229,0.833829,...,0.834679,0.832129,0.832129,0.795580,0.833404,0.833404,0.834254,0.834679,0.835104,0.832979,0.776881,0.824904,0.83340

In [15]:
EmbFilePath = dirData + "Benchmark/Emb_BP_Euka.h5"
inData2 = pd.read_hdf(EmbFilePath)
inData2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
AB17B_MOUSE,-0.087698,-0.042044,-0.142738,0.076032,-0.010172,-0.124418,0.031549,-0.052435,-0.028867,-0.031947,0.029296,-0.123174,-0.070111,-0.049643,0.063246,0.101371,0.113565,0.059259,-0.016383,0.058751,0.071362,-0.034621,-0.003431,0.021609,-0.099431,0.083293,-0.042268,-0.017087,0.106015,-0.057353,-0.030151,0.120004,-0.021011,-0.032151,-0.090565,0.054364,0.031651,-0.083577,0.084970,-0.021922,...,0.069343,0.050899,0.038233,-0.018243,0.083259,0.016243,0.045920,0.035900,0.077050,-0.032045,0.071024,0.116627,0.122173,-0.114490,-0.128083,0.156884,0.058390,-0.137202,0.002440,0.055067,-0.045952,-0.000363,-0.027813,-0.006770,0.043545,0.059809,-0.028076,0.035403,-0.119735,-0.017868,0.048434,0.011286,-0.119562,-0.221147,0.115504,-0.172535,-0.026675,-0.015521,-0.162345,-0.055825
AB17C_MOUSE,-0.033374,-0.007570,-0.094278,0.082492,-0.032613,-0.104200,-0.059237,0.013373,0.041502,-0.053527,-0.006504,-0.115872,-0.036657,-0.083982,0.173730,0.077215,0.082758,0.068691,-0.015282,0.010259,0.089448,-0.113865,0.031033,0.008410,-0.077081,0.073934,-0.014867,-0.086720,0.122022,-0.116931,-0.089639,0.082044,-0.070835,-0.046982,-0.091756,0.004993,0.011388,-0.090255,0.075087,0.082513,...,0.107585,0.049450,-0.000087,0.008309,0.058291,-0.005375,0.068340,-0.034637,0.103849,-0.059852,0.073139,0.121335,0.158056,-0.164297,-0.056600,0.148896,0.065931,-0.112600,0.033497,0.032696,-0.018079,-0.036294,0.018003,0.015997,0.091944,0.069599,-0.080075,0.042068,-0.125094,-0.072451,0.025641,-0.011655,-0.115871,-0.236579,0.162707,-0.196040,-0.027725,-0.012942,-0.135077,-0.082106
ABHD4_MOUSE,-0.071416,-0.111335,-0.129712,0.006021,-0.094052,-0.149953,0.091222,-0.124610,-0.123191,0.046677,-0.013901,-0.021129,-0.002910,-0.068994,0.152221,0.134546,0.080013,-0.037438,0.001901,-0.041537,0.114514,-0.082585,0.008468,0.014884,-0.018764,0.034679,0.012495,-0.065247,0.114662,-0.167435,-0.034922,0.168302,0.001613,0.036188,-0.107018,0.100466,0.064112,-0.024191,0.169783,0.069187,...,0.101548,0.097599,-0.001692,-0.007045,-0.025713,-0.076805,0.128906,0.038560,0.082869,0.056752,0.144984,0.018055,0.156292,-0.126036,-0.206216,0.118130,0.057627,-0.163676,-0.009763,0.062265,-0.133068,0.094346,-0.101323,-0.106577,0.110447,-0.007973,-0.003826,0.063704,-0.090232,-0.114673,0.031299,0.078272,-0.078293,-0.097091,0.073202,-0.104448,0.003256,-0.049058,-0.072878,0.046521
ABHDD_MOUSE,-0.102275,-0.032009,-0.197966,0.073784,-0.009253,-0.085855,0.109043,-0.091667,-0.040165,-0.048966,-0.030138,-0.107593,-0.069748,-0.065874,0.063700,0.116970,0.092034,0.018009,-0.033424,-0.011703,0.050129,-0.019415,-0.019382,0.023386,-0.098805,0.081066,-0.023173,0.040293,0.104995,-0.002243,0.007952,0.135834,0.041693,-0.061712,-0.056631,0.006463,0.017528,-0.061939,0.075908,-0.003828,...,0.010736,0.092882,0.020507,-0.044181,0.089321,0.048307,0.064025,0.033722,0.026136,0.018544,0.082979,0.084136,0.154673,-0.144738,-0.036354,0.069900,0.028089,-0.111470,0.021528,-0.053682,-0.081327,0.076024,-0.048189,-0.039142,0.122969,0.102773,0.071620,0.132182,-0.080996,0.046208,0.067552,0.018883,-0.184525,-0.104950,0.001222,-0.096210,-0.004161,0.028244,-0.150359,-0.055710
ADPGK_MOUSE,0.044418,0.006028,-0.010290,0.010217,0.007213,-0.047875,-0.051736,0.082782,0.118333,-0.023534,0.093585,-0.064750,0.024682,-0.001818,0.163361,-0.025318,0.037837,-0.000178,-0.022945,-0.042075,0.090387,-0.105366,0.018015,0.032266,0.049427,-0.014582,-0.063891,0.003330,0.002763,-0.009527,0.034610,0.030652,-0.099320,-0.096378,-0.078709,-0.178930,0.017671,0.080516,0.000633,0.169788,...,-0.032180,-0.069391,0.027342,0.007391,-0.041445,-0.041725,0.113566,-0.099046,-0.005068,-0.037854,-0.069007,0.011010,0.015957,-0.107710,-0.043521,0.005682,0.068597,-0.091653,-0.066884,0.157996,0.096368

In [16]:
TaxonFilePath = dirData + "Benchmark/Taxon_BP_Euka.h5"
inData3 = pd.read_hdf(TaxonFilePath)
inData3.head()

Taxon,2762,2769,2903,3037,3039,3055,3311,3469,3490,3498,3527,3562,3589,3627,3635,3649,3661,3663,3702,3707,3708,3712,3750,3760,3837,3847,3860,3878,3879,3880,3885,3886,3888,3917,3981,3988,4039,4058,4072,4081,...,298137,299727,317363,326986,330879,332648,336074,339422,341676,342590,356346,361634,367110,370292,373036,387623,389514,390785,394106,420151,425011,431936,432359,432528,446489,500485,510516,555479,559292,696368,746128,756487,756488,763456,1077530,1108046,1176036,1221240,1234705,1260784
Proteins,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AB17B_MOUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AB17C_MOUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ABHD4_MOUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ABHDD_MOUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ADPGK_MOUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
netOutFilePath = dirData + "Benchmark/netOut_BP_Euka.h5"

outData = pd.read_hdf(netOutFilePath)
outData.head()

Terms,GO:0000002,GO:0000003,GO:0000011,GO:0000018,GO:0000019,GO:0000023,GO:0000025,GO:0000027,GO:0000028,GO:0000038,GO:0000041,GO:0000042,GO:0000045,GO:0000050,GO:0000070,GO:0000075,GO:0000076,GO:0000077,GO:0000079,GO:0000082,GO:0000083,GO:0000086,GO:0000096,GO:0000097,GO:0000098,GO:0000103,GO:0000105,GO:0000122,GO:0000128,GO:0000132,GO:0000154,GO:0000160,GO:0000162,GO:0000165,GO:0000183,GO:0000184,GO:0000187,GO:0000188,GO:0000209,GO:0000212,...,GO:2001025,GO:2001038,GO:2001040,GO:2001056,GO:2001057,GO:2001141,GO:2001179,GO:2001180,GO:2001204,GO:2001206,GO:2001212,GO:2001213,GO:2001214,GO:2001222,GO:2001223,GO:2001224,GO:2001233,GO:2001234,GO:2001235,GO:2001236,GO:2001237,GO:2001238,GO:2001242,GO:2001243,GO:2001244,GO:2001251,GO:2001252,GO:2001257,GO:2001273,GO:2001275,GO:2001279,GO:2001280,GO:2001293,GO:2001295,GO:2001305,GO:2001307,GO:2001308,GO:2001310,GO:2001316,GO:2001317
AB17B_MOUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AB17C_MOUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ABHD4_MOUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ABHDD_MOUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ADPGK_MOUSE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
inData1 = torch.from_numpy(inData1.to_numpy())
inData2 = torch.from_numpy(inData2.to_numpy())
inData3 = torch.from_numpy(inData3.to_numpy())
outData = torch.from_numpy(outData.to_numpy())
if use_GPU:
    inData1 = inData1.cuda()
    inData2 = inData2.cuda()
    inData3 = inData3.cuda()
    outData = outData.cuda()

### 4. Model testing

In [19]:
_, f_Max, p_F_Max, r_F_Max, t_F_Max, _, _,_ = DeeProtGO_Euka_NK_BP.test(outData.float(), inData1, inData2,
                                                                        inData3, propPrediction = True, 
                                                                        CAFAerror = True)

In [20]:
f_Max, p_F_Max, r_F_Max, t_F_Max

(0.35238001204346453, 0.3989690172986025, 0.3155340045609746, 0.16)